In [1]:
import logging
import os
from fetch_data import (
    fetch_thl_cases_erva_daily, construct_cases_age_erva_daily,
    static_population_erva_age, construct_finland_age_cases_daily,
    compartment_values_daily, full_epidemic_state_finland
)
import pandas as pd
import numpy as np

curr_dir = os.getcwd()
stats_dir = os.path.join(curr_dir, 'stats')
logger = logging.getLogger(__name__)
curr_dir

'/Users/aponcedeleonch/Documents/Aalto/Courses/MasterThesis/Code/jeta'

In [2]:
age_file = os.path.join(stats_dir, 'erva_population_age_2020.csv')
pop_age, pop_age_prop = static_population_erva_age(logger, age_file)
pop_age

,erva,age_group,Total
0,All,0-9,547942
1,All,10-19,611151
2,All,20-29,661400
3,All,30-39,718761
4,All,40-49,666421
...,...,...,...
58,Åland,40-49,3853
59,Åland,50-59,4158
60,Åland,60-69,3896
61,Åland,70-79,3243


In [3]:
erva_cases = fetch_thl_cases_erva_daily(logger)

In [4]:
erva_cases.loc[erva_cases['Time'] >= '2021-01-04', ]

,Time,erva,val
2584,2021-01-04,HYKS,225.0
2585,2021-01-04,KYS,12.0
2586,2021-01-04,OYS,37.0
2587,2021-01-04,TAYS,38.0
2588,2021-01-04,TYKS,82.0
...,...,...,...
5112,2021-12-31,KYS,0.0
5113,2021-12-31,OYS,0.0
5114,2021-12-31,TAYS,0.0
5115,2021-12-31,TYKS,0.0


In [5]:
age, age_prop = construct_finland_age_cases_daily(logger)

In [6]:
age_prop.loc[age_prop['Time'] >= '2021-01-04', ]

,Time,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80+
371,2021-01-04,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
372,2021-01-05,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
373,2021-01-06,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
374,2021-01-07,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
375,2021-01-08,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
...,...,...,...,...,...,...,...,...,...,...
730,2021-12-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
731,2021-12-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
732,2021-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
733,2022-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
age_days = construct_cases_age_erva_daily(logger)

In [8]:
age_days.loc[age_days['Time'] >= '2021-01-04', ]

,Time,erva,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80+
2214,2021-01-04,HYKS,11.041439,24.413850,56.434024,43.552345,34.964558,26.499455,14.599237,7.606325,5.888768
2215,2021-01-04,KYS,0.588877,1.302072,3.009815,2.322792,1.864776,1.413304,0.778626,0.405671,0.314068
2216,2021-01-04,OYS,1.815703,4.014722,9.280262,7.161941,5.749727,4.357688,2.400763,1.250818,0.968375
2217,2021-01-04,TAYS,1.864776,4.123228,9.531080,7.355507,5.905125,4.475463,2.465649,1.284624,0.994547
2218,2021-01-04,TYKS,4.023991,8.897492,20.567067,15.872410,12.742639,9.657579,5.320611,2.772083,2.146129
...,...,...,...,...,...,...,...,...,...,...,...
4381,2021-12-31,KYS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4382,2021-12-31,OYS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4383,2021-12-31,TAYS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4384,2021-12-31,TYKS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
comp = compartment_values_daily(logger, age_file)

In [10]:
comp.loc[comp['date'] >= '2021-01-04', ]

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered
369,2021-01-04,HYKS,0-9,207567.673647,69.976068,699.760684,769.736752,657.092879,12618.496722
370,2021-01-05,HYKS,0-9,207475.62692,64.133747,641.337472,705.471219,627.683772,12804.218089
371,2021-01-06,HYKS,0-9,207404.876654,59.735716,597.357163,657.092879,608.826622,12942.203844
372,2021-01-07,HYKS,0-9,207309.544114,57.062161,570.621611,627.683772,638.842914,13036.929201
373,2021-01-08,HYKS,0-9,207244.438291,55.347875,553.478747,608.826622,605.704462,13154.030625
...,...,...,...,...,...,...,...,...,...
726,2021-12-27,Åland,80+,1698.50724,0.0,0.0,0.0,0.0,13.49276
727,2021-12-28,Åland,80+,1698.50724,0.0,0.0,0.0,0.0,13.49276
728,2021-12-29,Åland,80+,1698.50724,0.0,0.0,0.0,0.0,13.49276
729,2021-12-30,Åland,80+,1698.50724,0.0,0.0,0.0,0,13.49276


In [11]:
epidemic_state = full_epidemic_state_finland(logger, age_file)

In [12]:
epidemic_state.loc[epidemic_state['date'] >= '2021-01-04', ]

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered,First dose,Second dose,First dose cumulative,Second dose cumulative
369,2021-01-04,HYKS,0-9,207567.673647,69.976068,699.760684,769.736752,657.092879,12618.496722,0,0,0,0
370,2021-01-05,HYKS,0-9,207475.626920,64.133747,641.337472,705.471219,627.683772,12804.218089,0,0,0,0
371,2021-01-06,HYKS,0-9,207404.876654,59.735716,597.357163,657.092879,608.826622,12942.203844,0,0,0,0
372,2021-01-07,HYKS,0-9,207309.544114,57.062161,570.621611,627.683772,638.842914,13036.929201,0,0,0,0
373,2021-01-08,HYKS,0-9,207244.438291,55.347875,553.478747,608.826622,605.704462,13154.030625,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39469,2021-12-27,Åland,80+,141.507240,0.000000,0.000000,0.000000,0.000000,13.492760,0,0,1557,281
39470,2021-12-28,Åland,80+,141.507240,0.000000,0.000000,0.000000,0.000000,13.492760,0,0,1557,281
39471,2021-12-29,Åland,80+,141.507240,0.000000,0.000000,0.000000,0.000000,13.492760,0,0,1557,281
39472,2021-12-30,Åland,80+,141.507240,0.000000,0.000000,0.000000,0.000000,13.492760,0,0,1557,281
